In [1]:
%pip install nltk numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import requests

In [14]:


def scraping(url):
    # Define browser-like headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

    try:
        # Sending request to get the data from the webpage
        scraped_data = requests.get(url, headers=headers)
        scraped_data.raise_for_status()  # Check if request was successful (status code 200)

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(scraped_data.content, 'html.parser')

        # Find all paragraphs and extract text
        paragraphs = soup.find_all('p')
        article_context = ""

        for p in paragraphs:
            article_context += p.get_text(strip=True) + " "  # Strip whitespace and concatenate paragraphs

        return article_context.strip()  # Return the article content, ensuring no leading/trailing whitespace

    except requests.exceptions.RequestException as e:
        # Handle any exception raised by requests
        print(f"An error occurred: {e}")
        return None

In [24]:
# Sample text document
text = """
Natural Language Processing (NLP) is a sub-field of artificial intelligence that focuses on the interaction between computers and humans through natural language.
The ultimate goal of NLP is to enable computers to understand, interpret, and respond to human language in a way that is both valuable and meaningful.
Applications of NLP are used in a wide range of industries including customer service, healthcare, finance, and more. NLP helps businesses automate repetitive tasks, analyze vast amounts of text data, and gain insights from customer interactions.
"""
url = "https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)"
text = scraping(url)
# Tokenize the document into sentences
sentences = sent_tokenize(text)

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

In [25]:

# Compute the average TF-IDF score for each sentence
sentence_scores = tfidf_matrix.mean(axis=1)

In [26]:
# writing a function to get the top scored sentences indices
def top_n_indices(arr, num_sentences = 10):
    # Flatten the array to 1D, get the indices of the top n values
    indices = np.argsort(arr.flatten())[-num_sentences:]
    return indices

In [27]:

# Sort sentences by their scores and select the top N sentences for the summary
num_sentences = 2  # Number of sentences for the summary
top_sentence_indices = top_n_indices(np.array(sentence_scores),num_sentences)

In [37]:
def WriteFile(final_summary):
    with open('../summary.txt',mode='a', encoding='utf-8') as file:
        file.write(f"\n \n \n The TF-ID summarized text using NLTK is as follows : \n {final_summary}")

In [38]:
# Sort the selected sentences by their order of appearance
sorted_top_sentence_indices = sorted(top_sentence_indices)
# Create the summary
summary = " ".join([sentences[i] for i in sorted_top_sentence_indices])
print("Summary:")
print(summary)
WriteFile(summary)

Summary:
[3] For some architectures, such asconvolutional neural networks, it is common to keep the earlier layers (those closest to the input layer) frozen, as they capture lower-levelfeatures, while later layers often discern high-level features that can be more related to the task that the model is trained on. Fine-tuning is common innatural language processing(NLP), especially in the domain oflanguage modeling.Large language modelslikeOpenAI's series ofGPT foundation modelscan be fine-tuned on data for specific downstream NLP tasks (tasks that use a pre-trained model) to improve performance over the unmodified pre-trained model.
